In [1]:
pip install spotipy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 250.3/250.3 kB 121.0 kB/s eta 0:00:0000:0100:01
Note: you may need to restart the kernel to use updated packages.


In [2]:
import spotipy
import pandas as pd
from spotipy.oauth2 import SpotifyClientCredentials
import requests
import time

In [3]:
client_id = "1c286d70a9b546329741719289b46735"
client_secret = "d13ec7222b9a4f0c99fb513d1c6d9ff6"
client_credentials_manager = SpotifyClientCredentials(client_id=client_id, client_secret=client_secret)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

In [4]:
# Getting Access
AUTH_URL = 'https://accounts.spotify.com/api/token'
# POST
auth_response = requests.post(AUTH_URL, {
    'grant_type': 'client_credentials',
    'client_id': client_id,
    'client_secret': client_secret,
})

# convert the response to JSON
auth_response_data = auth_response.json()

# save the access token
access_token = auth_response_data['access_token']

In [5]:
headers = {
    'Authorization': 'Bearer {token}'.format(token=access_token)
}

In [6]:
# base URL of all Spotify API endpoints
BASE_URL = 'https://api.spotify.com/v1/'

# your artist ID - The Weeknd
artist_id = '06HL4z0CvFAxyc27GXpf02' #taylor swift
# extract all albums
r = requests.get(BASE_URL + 'artists/' + artist_id + '/albums',
                 headers=headers,
                 params={'include_groups': 'album', 'limit': 50})
d = r.json()

In [7]:
trackinfo = []   # will hold all track info
albums = [] # to keep track of duplicates

# loop over albums and get all tracks
for album in d['items']:
    album_name = album['name']


    trim_name = album_name.split('(')[0].strip()
    if trim_name.upper() in albums:
        continue
    albums.append(trim_name.upper()) # use upper() to standardize

    # this takes a few seconds so let's keep track of progress
    print(album_name)

    # pull all tracks from this album
    r = requests.get(BASE_URL + 'albums/' + album['id'] + '/tracks',
        headers=headers)
    tracks = r.json()['items']

    for track in tracks:
        # get track information
        f = requests.get(BASE_URL + 'tracks/' + track['id'],
            headers=headers)
        f = f.json()

     # combine with album info
        f.update({
            'album_name': album_name,
            'release_date': album['release_date'],
            'album_id': album['id']
        })

        trackinfo.append(f)

1989 (Taylor's Version) [Deluxe]
Speak Now (Taylor's Version)
Midnights (The Til Dawn Edition)
Red (Taylor's Version)
Fearless (Taylor's Version)
evermore (deluxe version)
folklore: the long pond studio sessions (from the Disney+ special) [deluxe edition]
folklore (deluxe version)
Lover
reputation
reputation Stadium Tour Surprise Song Playlist
Speak Now World Tour Live
Live From Clear Channel Stripped 2008
Taylor Swift


In [8]:
df = pd.DataFrame(trackinfo)
df.head()

,album,artists,available_markets,disc_number,duration_ms,explicit,external_ids,external_urls,href,id,is_local,name,popularity,preview_url,track_number,type,uri,album_name,release_date,album_id
0,"{'album_type': 'album', 'artists': [{'external...",[{'external_urls': {'spotify': 'https://open.s...,"[AR, AU, AT, BE, BO, BR, BG, CA, CL, CO, CR, C...",1,212600,False,{'isrc': 'USUG12306672'},{'spotify': 'https://open.spotify.com/track/4W...,https://api.spotify.com/v1/tracks/4WUepByoeqce...,4WUepByoeqcedHoYhSNHRt,False,Welcome To New York (Taylor's Version),73,None,1,track,spotify:track:4WUepByoeqcedHoYhSNHRt,1989 (Taylor's Version) [Deluxe],2023-10-27,1o59UpKw81iHR0HPiSkJR0
1,"{'album_type': 'album', 'artists': [{'external...",[{'external_urls': {'spotify': 'https://open.s...,"[AR, AU, AT, BE, BO, BR, BG, CA, CL, CO, CR, C...",1,231833,False,{'isrc': 'USUG12306673'},{'spotify': 'https://open.spotify.com/track/01...,https://api.spotify.com/v1/tracks/0108kcWLnn2H...,0108kcWLnn2HlH2kedi1gn,False,Blank Space (Taylor's Version),74,None,2,track,spotify:track:0108kcWLnn2HlH2kedi1gn,1989 (Taylor's Version) [Deluxe],2023-10-27,1o59UpKw81iHR0HPiSkJR0
2,"{'album_type': 'album', 'artists': [{'external...",[{'external_urls': {'spotify': 'https://open.s...,"[AR, AU, AT, BE, BO, BR, BG, CA, CL, CO, CR, C...",1,231000,False,{'isrc': 'USUG12306674'},{'spotify': 'https://open.spotify.com/track/3V...,https://api.spotify.com/v1/tracks/3Vpk1hfMAQme...,3Vpk1hfMAQme8VJ0SNRSkd,False,Style (Taylor's Version),75,None,3,track,spotify:track:3Vpk1hfMAQme8VJ0SNRSkd,1989 (Taylor's Version) [Deluxe],2023-10-27,1o59UpKw81iHR0HPiSkJR0
3,"{'album_type': 'album', 'artists': [{'external...",[{'external_urls': {'spotify': 'https://open.s...,"[AR, AU, AT, BE, BO, BR, BG, CA, CL, CO, CR, C...",1,235800,False,{'isrc': 'USUG12306675'},{'spotify': 'https://open.spotify.com/track/1O...,https://api.spotify.com/v1/tracks/1OcSfkeCg9hR...,1OcSfkeCg9hRC2sFKB4IMJ,False,Out Of The Woods (Taylor's Version),74,None,4,track,spotify:track:1OcSfkeCg9hRC2sFKB4IMJ,1989 (Taylor's Version) [Deluxe],2023-10-27,1o59UpKw81iHR0HPiSkJR0
4,"{'album_type': 'album', 'artists': [{'external...",[{'external_urls': {'spotify': 'https://open.s...,"[AR, AU, AT, BE, BO, BR, BG, CA, CL, CO, CR, C...",1,193289,False,{'isrc': 'USUG12306676'},{'spotify': 'https://open.spotify.com/track/2k...,https://api.spotify.com/v1/tracks/2k0ZEeAqzvYM...,2k0ZEeAqzvYMcx9Qt5aClQ,False,All You Had To Do Was Stay (Taylor's Version),73,None,5,track,spotify:track:2k0ZEeAqzvYMcx9Qt5aClQ,1989 (Taylor's Version) [Deluxe],2023-10-27,1o59UpKw81iHR0HPiSkJR0


In [9]:
df = df[['album_id','album_name', 'id', 'name', 'release_date', 'duration_ms', 'track_number', 'popularity', 'explicit', 'uri']]
df.head()

,album_id,album_name,id,name,release_date,duration_ms,track_number,popularity,explicit,uri
0,1o59UpKw81iHR0HPiSkJR0,1989 (Taylor's Version) [Deluxe],4WUepByoeqcedHoYhSNHRt,Welcome To New York (Taylor's Version),2023-10-27,212600,1,73,False,spotify:track:4WUepByoeqcedHoYhSNHRt
1,1o59UpKw81iHR0HPiSkJR0,1989 (Taylor's Version) [Deluxe],0108kcWLnn2HlH2kedi1gn,Blank Space (Taylor's Version),2023-10-27,231833,2,74,False,spotify:track:0108kcWLnn2HlH2kedi1gn
2,1o59UpKw81iHR0HPiSkJR0,1989 (Taylor's Version) [Deluxe],3Vpk1hfMAQme8VJ0SNRSkd,Style (Taylor's Version),2023-10-27,231000,3,75,False,spotify:track:3Vpk1hfMAQme8VJ0SNRSkd
3,1o59UpKw81iHR0HPiSkJR0,1989 (Taylor's Version) [Deluxe],1OcSfkeCg9hRC2sFKB4IMJ,Out Of The Woods (Taylor's Version),2023-10-27,235800,4,74,False,spotify:track:1OcSfkeCg9hRC2sFKB4IMJ
4,1o59UpKw81iHR0HPiSkJR0,1989 (Taylor's Version) [Deluxe],2k0ZEeAqzvYMcx9Qt5aClQ,All You Had To Do Was Stay (Taylor's Version),2023-10-27,193289,5,73,False,spotify:track:2k0ZEeAqzvYMcx9Qt5aClQ


In [10]:
audiofeature = []   # will hold all audio feature info
albums = [] # to keep track of duplicates

# loop over albums and get all tracks
for album in d['items']:
    album_name = album['name']

    # here's a hacky way to skip over albums we've already grabbed
    trim_name = album_name.split('(')[0].strip()
    if trim_name.upper() in albums:
        continue
    albums.append(trim_name.upper()) # use upper() to standardize

    # this takes a few seconds so let's keep track of progress
    print(album_name)

    # pull all tracks from this album
    r = requests.get(BASE_URL + 'albums/' + album['id'] + '/tracks',
        headers=headers)
    tracks = r.json()['items']

    for track in tracks:
        # get audio features (key, liveness, danceability, ...)
        f = requests.get(BASE_URL + 'audio-features/' + track['id'],
            headers=headers)
        f = f.json()

        # combine with album info
        f.update({
            'track_name': track['name'],
            'album_name': album_name,
            'release_date': album['release_date'],
            'album_id': album['id']
        })

        audiofeature.append(f)

1989 (Taylor's Version) [Deluxe]
Speak Now (Taylor's Version)
Midnights (The Til Dawn Edition)
Red (Taylor's Version)
Fearless (Taylor's Version)
evermore (deluxe version)
folklore: the long pond studio sessions (from the Disney+ special) [deluxe edition]
folklore (deluxe version)
Lover
reputation
reputation Stadium Tour Surprise Song Playlist
Speak Now World Tour Live
Live From Clear Channel Stripped 2008
Taylor Swift


In [11]:
df2 = pd.DataFrame(audiofeature)
df2.head()

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,...,id,uri,track_href,analysis_url,duration_ms,time_signature,track_name,album_name,release_date,album_id
0,0.757,0.610,7,-4.840,1,0.0327,0.009420,0.000037,0.3670,0.685,...,4WUepByoeqcedHoYhSNHRt,spotify:track:4WUepByoeqcedHoYhSNHRt,https://api.spotify.com/v1/tracks/4WUepByoeqce...,https://api.spotify.com/v1/audio-analysis/4WUe...,212600,4,Welcome To New York (Taylor's Version),1989 (Taylor's Version) [Deluxe],2023-10-27,1o59UpKw81iHR0HPiSkJR0
1,0.733,0.733,0,-5.376,1,0.0670,0.088500,0.000000,0.1680,0.701,...,0108kcWLnn2HlH2kedi1gn,spotify:track:0108kcWLnn2HlH2kedi1gn,https://api.spotify.com/v1/tracks/0108kcWLnn2H...,https://api.spotify.com/v1/audio-analysis/0108...,231833,4,Blank Space (Taylor's Version),1989 (Taylor's Version) [Deluxe],2023-10-27,1o59UpKw81iHR0HPiSkJR0
2,0.511,0.822,11,-4.785,0,0.0397,0.000421,0.019700,0.0899,0.305,...,3Vpk1hfMAQme8VJ0SNRSkd,spotify:track:3Vpk1hfMAQme8VJ0SNRSkd,https://api.spotify.com/v1/tracks/3Vpk1hfMAQme...,https://api.spotify.com/v1/audio-analysis/3Vpk...,231000,4,Style (Taylor's Version),1989 (Taylor's Version) [Deluxe],2023-10-27,1o59UpKw81iHR0HPiSkJR0
3,0.545,0.885,0,-5.968,1,0.0447,0.000537,0.000056,0.3850,0.206,...,1OcSfkeCg9hRC2sFKB4IMJ,spotify:track:1OcSfkeCg9hRC2sFKB4IMJ,https://api.spotify.com/v1/tracks/1OcSfkeCg9hR...,https://api.spotify.com/v1/audio-analysis/1OcS...,235800,4,Out Of The Woods (Taylor's Version),1989 (Taylor's Version) [Deluxe],2023-10-27,1o59UpKw81iHR0HPiSkJR0
4,0.588,0.721,0,-5.579,1,0.0317,0.000656,0.000000,0.1310,0.520,...,2k0ZEeAqzvYMcx9Qt5aClQ,spotify:track:2k0ZEeAqzvYMcx9Qt5aClQ,https://api.spotify.com/v1/tracks/2k0ZEeAqzvYM...,https://api.spotify.com/v1/audio-analysis/2k0Z...,193289,4,All You Had To Do Was Stay (Taylor's Version),1989 (Taylor's Version) [Deluxe],2023-10-27,1o59UpKw81iHR0HPiSkJR0


In [12]:
result = pd.merge(df, df2, on='id')
print(result)

                 album_id_x                      album_name_x  \
0    1o59UpKw81iHR0HPiSkJR0  1989 (Taylor's Version) [Deluxe]   
1    1o59UpKw81iHR0HPiSkJR0  1989 (Taylor's Version) [Deluxe]   
2    1o59UpKw81iHR0HPiSkJR0  1989 (Taylor's Version) [Deluxe]   
3    1o59UpKw81iHR0HPiSkJR0  1989 (Taylor's Version) [Deluxe]   
4    1o59UpKw81iHR0HPiSkJR0  1989 (Taylor's Version) [Deluxe]   
..                      ...                               ...   
241  5eyZZoQEFQWRHkV2xgAeBw                      Taylor Swift   
242  5eyZZoQEFQWRHkV2xgAeBw                      Taylor Swift   
243  5eyZZoQEFQWRHkV2xgAeBw                      Taylor Swift   
244  5eyZZoQEFQWRHkV2xgAeBw                      Taylor Swift   
245  5eyZZoQEFQWRHkV2xgAeBw                      Taylor Swift   

                         id                                           name  \
0    4WUepByoeqcedHoYhSNHRt         Welcome To New York (Taylor's Version)   
1    0108kcWLnn2HlH2kedi1gn                 Blank Space (Taylor

In [13]:
result.to_csv("myData1.csv", sep = ',')